# 第3部 第7章 正規線形モデル

## はじめに

## フォルダ構造とユーティリティ関数、ライブラリimport
リンク集の記事にフォルダ構造とユーティリティ関数、ライブラリimportを掲載しました。\
準備としてそちらのページをご覧ください。
1. [フォルダ構造とユーティリティ関数]()
1. [ライブラリimport]()

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
cmap = plt.get_cmap("tab10")
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 7.1 本章の目的と概要
参考書籍から引用します。\
RとStanやbrmsの部分をPythonとNumPyroに読み替えてください。

> **テーマ**\
> 本章では量的データや質的データを問わず,複数の説明変数を持ち,応答変数が正規分布に従うモデルについて解説します.\
> これは正規線形モデルとも呼ばれます.\
> 第3部第2章から単回帰モデルを学び始めて,本章が1つのまとめになります.

> **目的**\
> 簡単な構造を組み合わせて複雑なモデルを構築する考え方を学んでいただくために,本章を執筆しました.\
> 本章で新たに登場する概念はありません.\
> 今まで学んできた構造だけを組み合わせて,複雑なモデルを推定します.

> **概要**\
> モデルの構造 → 分析の準備 → データの読み込みと可視化\
> → brms による正規線形モデルの推定 → (補足)正規線形モデルのデザイン行列

## 7.2 モデルの構造
ビールの売り上げ $y_{i}$ と天気 $( x_{i, 1}, x_{i, 2} )$ と気温 $x_{i, 3}$ の関係を数式で整理します。\
参考書籍では次のように書かれています。\
ここで $x_{i,1}$ は晴れの日に $1$ , そうでない場合に $0$ を取ります。\
$x_{i,2}$ は雨の日に $1$ , そうでない場合に $0$ を取ります。\
$( x_{i, 1}, x_{i, 2} ) = (0, 0)$ の場合は曇りを表します。

$$
\begin{aligned}
\mu_{i} =& \beta_{0} + \beta_{1} x_{i, 1} + \beta_{2} x_{i, 2} + \beta_{3} x_{i, 3}\\
y_{i} \sim& Normal(\mu_{i}, \sigma^2)
\end{aligned}
$$

回帰分析と同様に確率モデルのパラメータの事前分布からモデル化します。

## 7.3 分析の準備
ライブラリの準備をしている節です。\
省略します。

## 6.4 データの読み込みと可視化
データを読み込んで概要を把握します。

In [ ]:
# データを読み込む
df = pl.read_csv("../data/3-7-1-beer-sales-4.csv")
display(df.head())

# サンプルサイズを表示する
print( len(df) )

天気別のビールの売り上げと気温の散布図をプロットします。

In [ ]:
ax = sns.scatterplot(data = df, x = "temperature", y = "sales", hue = "weather")
ax.set_title("図3.7.1 ビールの売り上げと天気・気温の散布図")

モデルの構造の通りのダミー変数を作成します.

In [ ]:
df = (
    df
    .with_columns([
        pl.when( pl.col("weather") == "sunny" ).then(1)
        .otherwise(0)
        .alias("x1"),
        pl.when( pl.col("weather") == "rainy" ).then(1)
        .otherwise(0)
        .alias("x2"),
    ])
)
display(df.head())

numpyroに渡すためにjax.numpy配列に変換します。

In [ ]:
X1 = jnp.array(df["x1"], dtype = int)
X2 = jnp.array(df["x2"], dtype = int)
X3 = jnp.array(df["temperature"], dtype = float)
Y = jnp.array(df["sales"], dtype = float)

## 7.5 brmsによる正規線形モデルの推定
確率モデルを作成します。

In [ ]:
def model_normal_linear(X1, X2, X3, Y = None):
    '''
        第3部 7章 のビールの売り上げ
    '''
    # 説明変数をモデルに明示する
    # ベクトル化(学習用データを確率変数に割り当てるためのNumPyroのお作法)
    N = len(X1)
    with numpyro.plate("N", N):
        X1 = numpyro.deterministic("X1", X1)
        X2 = numpyro.deterministic("X2", X2)
        X3 = numpyro.deterministic("X3", X3)
    # 確率変数のパラメータの事前分布を設定する
    β0 = numpyro.sample("β0", dist.Normal(loc = 0, scale = 100))
    β1 = numpyro.sample("β1", dist.Normal(loc = 0, scale = 50))
    β2 = numpyro.sample("β2", dist.Normal(loc = 0, scale = 50))
    β3 = numpyro.sample("β3", dist.Normal(loc = 0, scale = 50))
    # リンク関数(恒等関数)を定義する
    μ = numpyro.deterministic("μ", β0 + β1 * X1 + β2 * X2 + β3 * X3)
    # 目的変数が従う正規分布の標準偏差の事前分布を設定する
    σ = numpyro.sample("σ", dist.HalfNormal(scale = 30))
    # 目的変数 y は説明変数 x の値に応じた平均値 μ をパラメータとする正規分布に従うと仮定します。
    with numpyro.plate("N", N):
        numpyro.sample("Y", dist.Normal(loc = μ, scale = σ), obs = Y)

モデルをプロットします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "X3": X3,
    "Y": Y,
}
try_render_model(model_normal_linear, render_name = "ビールの売り上げと天気・気温の関係", **model_args)

ユーティリティ関数にモデルを渡してサンプリングします。

In [ ]:
model_args = {
    "X1": X1,
    "X2": X2,
    "X3": X3,
    "Y": Y
}
mcmc = run_mcmc(
    model_normal_linear,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.9,
    **model_args
)
idata = az.from_numpyro(mcmc, log_likelihood = False)

サンプリングが上手くいったか確認します。

In [ ]:
az.plot_trace(idata, compact = False, var_names = ["β0", "β1", "β2", "β3", "σ"])
plt.tight_layout()

MCMCサンプルの概要を確認します。\
HDIを見ると $\beta 1$ は明確に正の値を取り、 $\beta 2$ は0をまたいでいます。

In [ ]:
az.summary(idata, var_names = ["β0", "β1", "β2", "β3"])

天気別の事後予測分布を計算します。\
NumPyroは少し準備が必要です。

In [ ]:
# 予測用のデータ
list_temperature = [df["temperature"].floor().min(), df["temperature"].ceil().max()]
X3 = jnp.array(list_temperature * 3, dtype = float)
model_args = {
    "X1": jnp.array([0, 0] + [1, 1] + [0, 0], dtype = int),
    "X2": jnp.array([0, 0] + [0, 0] + [1, 1], dtype = int),
    "X3": X3,
    "Y": None,
}
# 予測データの目的変数名
var_name = "Y"

# 事後予測分布を計算する
idata = compute_posterior_predictive_distribution(
    model_normal_linear,
    mcmc,
    var_name = "Y",
    seed = 42,
    hdi_prob = 0.95,
    log_likelihood = False,
    **model_args,
)

# 事後予測平均
y_ppc_mean = np.array( idata["posterior_predictive"][var_name].mean(dim = ("chain", "draw")) )
# 事後予測のHDI
hdi_ppc = az.hdi(
    idata,                          # InferenceData 全体を渡す
    group = "posterior_predictive", # 計算対象グループを指定
    var_names = [var_name],         # 対象変数
    hdi_prob = 0.95,            # 区間幅
)
y_ppc_low = np.array( hdi_ppc[var_name][:, 0] )
y_ppc_high = np.array( hdi_ppc[var_name][:, 1] )

# データを保存する
df_pred = pl.DataFrame({
    "weather": ["cloudy"] * 2 + ["sunny"] * 2 + ["rainy"] * 2,
    "temperature": np.array(X3),
    "y_pred": y_ppc_mean,
    "y_ppc_low": y_ppc_low,
    "y_ppc_high": y_ppc_high,
})

散布図と事後予測分布をプロットします。\
晴れの日の売り上げが好調であることがわかります。

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1)
for idx, weather in enumerate(df["weather"].unique().sort()):
    expr = pl.col("weather") == weather
    sns.scatterplot(data = df.filter(expr), x = "temperature", y = "sales", ax = ax, color = cmap(idx), label = weather)
    sns.lineplot(data = df_pred.filter(expr), x = "temperature", y = "y_pred", ax = ax, color = cmap(idx))
    ax.fill_between(
        x = df_pred.filter(expr)["temperature"], y1 = df_pred.filter(expr)["y_ppc_high"], y2 = df_pred.filter(expr)["y_ppc_low"],
        color = cmap(idx), alpha=0.2
    )
plt.suptitle("図3.7.2 正規線形モデルから得られた回帰直線")

## 7.6 補足: 正規線形モデルのデザイン行列
省略

## 終わりに